In [ ]:
from pdf2image import convert_from_path
import pytesseract
from transformers import pipeline

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


In [ ]:
pdf_file = "Bb.pdf"

In [ ]:
pages = convert_from_path(pdf_file, dpi=300)


In [ ]:
ocr_texts = []
for i, page in enumerate(pages):
    text = pytesseract.image_to_string(page, lang="eng")
    ocr_texts.append(text)


In [ ]:
document_text = "\n".join(ocr_texts)

In [ ]:
llm = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
prompt = f"""
You are given OCR extracted text from the Indian Contract Act, 1872:

---
{document_text}
---

Your task:
1. Clean the OCR noise (fix spelling mistakes, remove garbage text).
2. Identify and extract important **legal terms and definitions** from the Act, such as:
   - Proposal, Acceptance, Promise, Agreement, Consideration, Contract
   - Void Contract, Voidable Contract, Illegal Agreement
   - Free Consent, Coercion, Undue Influence, Fraud, Misrepresentation, Mistake
   - Competency of Parties, Lawful Consideration, Lawful Object
   - Breach of Contract, Remedies, Damages, Quantum Meruit
   - Indemnity, Guarantee, Bailment, Pledge, Agency
3. For each term, give:
   - **Term Name**
   - **Definition (in simple language from the text)**
   - **Section Number (if available in the Act)**
4. Ignore irrelevant or noisy text.

Return the output as a structured JSON list like this:
[
  {{ "term": "Contract", "definition": "...", "section": "Section 2(h)" }},
  {{ "term": "Consideration", "definition": "...", "section": "Section 2(d)" }}
]
"""


In [ ]:
response = llm(prompt, max_new_tokens=100, do_sample=True, temperature=0.2)
output_text = response[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [ ]:
with open("filtered_output.txt", "w", encoding="utf-8") as f:
    f.write(output_text)

print("✅ Results saved in filtered_output.txt")

✅ Results saved in filtered_output.txt
